<a href="https://colab.research.google.com/github/Asfara17/ASFARA/blob/main/Deepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn nltk transformers
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
import string

# Download the necessary NLTK data
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import files

# Upload the files
uploaded = files.upload()

# Check the filenames of the uploaded files
print(uploaded.keys())  # This will print the exact names of the uploaded files

# Load the 'real' news CSV and label them as 'real'
# Use the exact filenames shown in the print statement above
df_real = pd.read_csv(list(uploaded.keys())[0])  # Assuming the first uploaded file is 'real.csv'
df_real['label'] = 'real'  # Add a label column for real news

# Load the 'fake' news CSV and label them as 'fake'
df_fake = pd.read_csv(list(uploaded.keys())[1])  # Assuming the second uploaded file is 'fake.csv'
df_fake['label'] = 'fake'  # Add a label column for fake news

# Combine both DataFrames into one
df = pd.concat([df_real, df_fake], ignore_index=True)

# Check the first few rows of the combined dataset
df.head()

In [ ]:
from google.colab import files

# Upload the files
uploaded = files.upload()

# Check the filenames of the uploaded files
print(uploaded.keys())  # This will print the exact names of the uploaded files

# Load the 'real' news CSV and label them as 'real'
# Use the exact filenames shown in the print statement above
df_real = pd.read_csv(list(uploaded.keys())[0])  # Assuming the first uploaded file is 'real.csv'
df_real['label'] = 'real'  # Add a label column for real news

# Load the 'fake' news CSV and label them as 'fake'
df_fake = pd.read_csv(list(uploaded.keys())[1])  # Assuming the second uploaded file is 'fake.csv'
df_fake['label'] = 'fake'  # Add a label column for fake news

# Combine both DataFrames into one
df = pd.concat([df_real, df_fake], ignore_index=True)

# Check the first few rows of the combined dataset
df.head()

In [ ]:
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import string

# Download NLTK data for stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

# Preprocessing function to clean the text
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    text = text.lower()  # Convert to lowercase
    text = ''.join([char for char in text if char not in string.punctuation])  # Remove punctuation
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# Apply the preprocessing function to the text column
df['cleaned_text'] = df['text'].apply(preprocess_text)

# Check the cleaned data
df.head()

In [ ]:
# Features (text data) and target (labels: 'real' or 'fake')
X = df['cleaned_text']
y = df['label']

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the split data sizes
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the training data, transform the test data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Check the shape of the transformed data
print(f"Shape of training data: {X_train_tfidf.shape}")
print(f"Shape of test data: {X_test_tfidf.shape}")

# Initialize the Naive Bayes classifier
model = MultinomialNB()

# Train the model using the training data
model.fit(X_train_tfidf, y_train)

# Predict the labels for the test data
y_pred = model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print the classification report (Precision, Recall, F1-score)
print(classification_report(y_test, y_pred))

def detect_fake_news(news_article):
    # Preprocess the input article
    cleaned_article = preprocess_text(news_article)

    # Convert the cleaned text to TF-IDF
    article_tfidf = vectorizer.transform([cleaned_article])

    # Predict if it's fake or real
    prediction = model.predict(article_tfidf)

    return prediction[0]  # Return 'real' or 'fake'

# Example: Test with a new article
new_article = "Breaking: The government has just announced a new law to prevent fake news."
result = detect_fake_news(new_article)
print(f'The article is: {result}')

import joblib

# Save the model and vectorizer
joblib.dump(model, 'fake_news_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

# Later, you can load the model and vectorizer like this:
# model = joblib.load('fake_news_model.pkl')
# vectorizer = joblib.load('tfidf_vectorizer.pkl')

import joblib
import string
import nltk
from nltk.corpus import stopwords

# Download NLTK data for stopwords
nltk.download('stopwords')

# Load the trained model and vectorizer
model = joblib.load('fake_news_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Preprocessing function to clean the text
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    text = text.lower()  # Convert to lowercase
    text = ''.join([char for char in text if char not in string.punctuation])  # Remove punctuation
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# Function to predict whether the news is real or fake
def detect_fake_news(news_article):
    # Preprocess the input article
    cleaned_article = preprocess_text(news_article)

    # Convert the cleaned text to TF-IDF
    article_tfidf = vectorizer.transform([cleaned_article])

    # Predict if it's fake or real
    prediction = model.predict(article_tfidf)

    return prediction[0]  # Return 'real' or 'fake'

# Get user input for news article
user_input = input("Enter the news article text to check if it's real or fake: ")

# Use the function to detect if the news is fake or real
result = detect_fake_news(user_input)

# Output the result
if result == 'fake':
    print("The news is fake!")
else:
    print("The news is real!")